In [64]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

# 1. 데이터 로딩
df = pd.read_csv('데이터프레임_수정_v03.csv') 

# 2. 독립변수 / 종속변수 지정
X = df[['생활 인구', '소매업', '의료기관', '교육기관', '카드소비', '서비스업', '음식점', '지역면적(km^2)', '버스정류장수', '추정 차량등록수', '근린생활시설수']]
y = df['적정 주차면수']  # = 주차면수 합 + 단속건수*가중치

In [65]:
# XGBoost GridSearch
# 1. 모델과 파라미터 정의
xgb = XGBRegressor(random_state=0)
param_grid = {
    'n_estimators': [50, 60, 75, 90, 100, 150],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.03, 0.035, 0.04, 0.045, 0.05, 0.1],
    'subsample': [0.6, 0.8, 0.9]
}

# 2. R² 기반 GridSearchCV (CV 수행)
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='r2',
    cv=10,
    n_jobs=-1
)
grid_search.fit(X, y)
best_model = grid_search.best_estimator_

# 3. 교차검증 기반 성능 평가
n = X.shape[0]  # 표본 수
p = X.shape[1]  # 설명변수 수

# R²
cv_r2_rf = np.mean(cross_val_score(best_model, X, y, cv=10, scoring='r2'))

# Adjusted R²
adj_r2_rf = 1 - (1 - cv_r2_rf) * (n - 1) / (n - p - 1)

# RMSE
neg_mse_scores_rf = cross_val_score(best_model, X, y, cv=10, scoring='neg_mean_squared_error')
cv_rmse_rf = np.mean(np.sqrt(-neg_mse_scores_rf))

# MAE
neg_mae_scores_rf = cross_val_score(best_model, X, y, cv=10, scoring='neg_mean_absolute_error')
cv_mae_rf = np.mean(-neg_mae_scores_rf)

# 4. 결과 출력
print("✅ [XGBoost] 최적 하이퍼파라미터:", grid_search.best_params_)
print("📊 [XGBoost] 교차검증 기반 성능 지표:")
print(f"R²            : {cv_r2_rf:.5f}")
print(f"Adjusted R²   : {adj_r2_rf:.5f}")
print(f"RMSE          : {cv_rmse_rf:.5f}")
print(f"MAE           : {cv_mae_rf:.5f}")

✅ [XGBoost] 최적 하이퍼파라미터: {'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.6}
📊 [XGBoost] 교차검증 기반 성능 지표:
R²            : 0.59748
Adjusted R²   : 0.45912
RMSE          : 1886.03914
MAE           : 1404.68212


In [66]:
# RandomForest GridSearch
# 1. 모델과 파라미터 정의
rf = RandomForestRegressor(random_state=0)
param_grid_rf = {
    'n_estimators': [75, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}

# 2. GridSearchCV (R² 기준)
grid_search_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    scoring='r2',
    cv=10,
    n_jobs=-1
)
grid_search_rf.fit(X, y)
best_rf = grid_search_rf.best_estimator_

# 3. 교차검증 기반 성능 평가
n = X.shape[0]  # 표본 수
p = X.shape[1]  # 설명변수 수

# R²
cv_r2_rf = np.mean(cross_val_score(best_rf, X, y, cv=10, scoring='r2'))

# Adjusted R²
adj_r2_rf = 1 - (1 - cv_r2_rf) * (n - 1) / (n - p - 1)

# RMSE
neg_mse_scores_rf = cross_val_score(best_rf, X, y, cv=10, scoring='neg_mean_squared_error')
cv_rmse_rf = np.mean(np.sqrt(-neg_mse_scores_rf))

# MAE
neg_mae_scores_rf = cross_val_score(best_rf, X, y, cv=10, scoring='neg_mean_absolute_error')
cv_mae_rf = np.mean(-neg_mae_scores_rf)

# 4. 결과 출력
print("✅ [RandomForest] 최적 하이퍼파라미터:", grid_search_rf.best_params_)
print("📊 [RandomForest] 교차검증 기반 성능 지표:")
print(f"R²            : {cv_r2_rf:.5f}")
print(f"Adjusted R²   : {adj_r2_rf:.5f}")
print(f"RMSE          : {cv_rmse_rf:.5f}")
print(f"MAE           : {cv_mae_rf:.5f}")

✅ [RandomForest] 최적 하이퍼파라미터: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
📊 [RandomForest] 교차검증 기반 성능 지표:
R²            : 0.54705
Adjusted R²   : 0.39135
RMSE          : 1818.48173
MAE           : 1310.16325


In [67]:
# Catboost GridSearch
# 1. 모델과 파라미터 정의
cat = CatBoostRegressor(verbose=0, random_state=0)
param_grid_cat = {
    'iterations': [75, 100],
    'depth': [3, 4, 5],
    'learning_rate': [0.03, 0.05]
}

# 2. GridSearchCV (R² 기준)
grid_search_cat = GridSearchCV(
    estimator=cat,
    param_grid=param_grid_cat,
    scoring='r2',
    cv=10,
    n_jobs=-1
)
grid_search_cat.fit(X, y)
best_cat = grid_search_cat.best_estimator_

# 3. 교차검증 기반 성능 평가
n = X.shape[0]  # 표본 수
p = X.shape[1]  # 설명변수 수

# R²
cv_r2_cat = np.mean(cross_val_score(best_cat, X, y, cv=10, scoring='r2'))

# Adjusted R²
adj_r2_cat = 1 - (1 - cv_r2_cat) * (n - 1) / (n - p - 1)

# RMSE
neg_mse_scores_cat = cross_val_score(best_cat, X, y, cv=10, scoring='neg_mean_squared_error')
cv_rmse_cat = np.mean(np.sqrt(-neg_mse_scores_cat))

# MAE
neg_mae_scores_cat = cross_val_score(best_cat, X, y, cv=10, scoring='neg_mean_absolute_error')
cv_mae_cat = np.mean(-neg_mae_scores_cat)

# 4. 결과 출력
print("✅ [CatBoost] 최적 하이퍼파라미터:", grid_search_cat.best_params_)
print("📊 [CatBoost] 교차검증 기반 성능 지표:")
print(f"R²            : {cv_r2_cat:.5f}")
print(f"Adjusted R²   : {adj_r2_cat:.5f}")
print(f"RMSE          : {cv_rmse_cat:.5f}")
print(f"MAE           : {cv_mae_cat:.5f}")

✅ [CatBoost] 최적 하이퍼파라미터: {'depth': 3, 'iterations': 100, 'learning_rate': 0.05}
📊 [CatBoost] 교차검증 기반 성능 지표:
R²            : 0.56277
Adjusted R²   : 0.41247
RMSE          : 1901.71455
MAE           : 1419.78911


성능이 가장 좋은 XGBoost를 사용 모델로 채택합니다.

이후 변수제거법(Backward Feature Elimination)으로 특정 변수를 제거했을 때 성능이 올라가는지 확인합니다.  
하나씩 변수를 제거해가며, **교차검증 기반 성능(R², RMSE 등)**을 비교합니다.

In [68]:
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

# 전체 변수 목록
base_features = X.columns.tolist()

# 기준 모델
base_model = XGBRegressor(random_state=0)
base_r2 = np.mean(cross_val_score(base_model, X, y, cv=10, scoring='r2'))

results = []

for feature in base_features:
    reduced_features = [f for f in base_features if f != feature]
    X_reduced = X[reduced_features]
    
    model = XGBRegressor(random_state=0)
    r2 = np.mean(cross_val_score(model, X_reduced, y, cv=10, scoring='r2'))
    
    results.append({
        'Removed_Feature': feature,
        'CV_R2_after_removal': r2,
        'R2_Change': r2 - base_r2
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='R2_Change', ascending=False)

print("✅ 변수 제거 시 교차검증 R² 변화:")
print(results_df)

✅ 변수 제거 시 교차검증 R² 변화:
   Removed_Feature  CV_R2_after_removal  R2_Change
0            생활 인구             0.435435   1.310420
9         추정 차량등록수            -0.659121   0.215865
4             카드소비            -0.828747   0.046239
6              음식점            -0.834755   0.040231
3             교육기관            -0.856168   0.018817
8           버스정류장수            -0.860949   0.014037
2             의료기관            -0.868021   0.006965
1              소매업            -0.880768  -0.005782
10         근린생활시설수            -0.945267  -0.070281
5             서비스업            -0.999055  -0.124069
7       지역면적(km^2)            -1.250669  -0.375683


변수 제거시 R2값이 크게 오르는 생활인구를 제거하고 다시 XGBoost 진행

In [76]:
X = df[['소매업', '의료기관', '교육기관', '카드소비', '서비스업', '음식점', '지역면적(km^2)', '버스정류장수', '추정 차량등록수', '근린생활시설수']]

# 1. 최적 파라미터 적용
xgb_model = XGBRegressor(
    learning_rate=0.05,
    max_depth=3,
    n_estimators=50,
    subsample=0.9,
    random_state=0
)

# 2. 교차검증 기반 성능 평가
n = X.shape[0]  # 표본 수
p = X.shape[1]  # 설명변수 수

# R²
cv_r2 = np.mean(cross_val_score(xgb_model, X, y, cv=10, scoring='r2'))

# Adjusted R²
adj_r2 = 1 - (1 - cv_r2) * (n - 1) / (n - p - 1)

# RMSE
neg_mse_scores = cross_val_score(xgb_model, X, y, cv=10, scoring='neg_mean_squared_error')
cv_rmse = np.mean(np.sqrt(-neg_mse_scores))

# MAE
neg_mae_scores = cross_val_score(xgb_model, X, y, cv=10, scoring='neg_mean_absolute_error')
cv_mae = np.mean(-neg_mae_scores)

# 3. 결과 출력
print("📌 [XGBoost] 지정 파라미터:")
print("{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.9}")
print("📊 교차검증 기반 성능 지표:")
print(f"R²            : {cv_r2:.5f}")
print(f"Adjusted R²   : {adj_r2:.5f}")
print(f"RMSE          : {cv_rmse:.5f}")
print(f"MAE           : {cv_mae:.5f}")

📌 [XGBoost] 지정 파라미터:
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.9}
📊 교차검증 기반 성능 지표:
R²            : 0.62382
Adjusted R²   : 0.50983
RMSE          : 1800.20341
MAE           : 1238.72040


Optuna를 활용하여 최적 하이퍼파라미터를 튜닝합니다.

In [71]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        'n_estimators': 100,
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }
    model = XGBRegressor(**params, random_state=0)
    score = cross_val_score(model, X, y, cv=10, scoring='r2').mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)

print("Best params:", study.best_params)
print("Best CV R²:", study.best_value)

[I 2025-05-14 14:24:09,111] A new study created in memory with name: no-name-1048b741-0607-43a2-9257-9972e049f894
[I 2025-05-14 14:24:09,624] Trial 0 finished with value: 0.6069642129284359 and parameters: {'max_depth': 4, 'learning_rate': 0.04145080421553518, 'subsample': 0.8181284393022161, 'colsample_bytree': 0.7532479503980486, 'reg_alpha': 4.21573498729922, 'reg_lambda': 1.2254263648104473, 'min_child_weight': 1}. Best is trial 0 with value: 0.6069642129284359.
[I 2025-05-14 14:24:10,044] Trial 1 finished with value: 0.15419364838841498 and parameters: {'max_depth': 5, 'learning_rate': 0.07291749228012179, 'subsample': 0.6968282126200362, 'colsample_bytree': 0.953256718883746, 'reg_alpha': 0.367326317271654, 'reg_lambda': 2.134829105437768, 'min_child_weight': 9}. Best is trial 0 with value: 0.6069642129284359.
[I 2025-05-14 14:24:10,457] Trial 2 finished with value: 0.5962496216807714 and parameters: {'max_depth': 3, 'learning_rate': 0.027110463292952622, 'subsample': 0.603780176

Best params: {'max_depth': 5, 'learning_rate': 0.048663077640840165, 'subsample': 0.6703350473987629, 'colsample_bytree': 0.7037708069085559, 'reg_alpha': 3.005852508938722, 'reg_lambda': 1.2380253357332034, 'min_child_weight': 1}
Best CV R²: 0.7254083024291335


최적의 파라미터로 다시 XGBoost 학습을 진행합니다.

Best params: {'max_depth': 5, 'learning_rate': 0.048663077640840165, 'subsample': 0.6703350473987629, 'colsample_bytree': 0.7037708069085559, 'reg_alpha': 3.005852508938722, 'reg_lambda': 1.2380253357332034, 'min_child_weight': 1}  
Best CV R²: 0.7254083024291335  

In [72]:
# Optuna에서 얻은 최적 파라미터
best_params = study.best_params

# best_params에 추가적으로 고정 파라미터 (예: random_state) 포함해도 됨
best_model = XGBRegressor(**best_params, random_state=0)

# 2. 교차검증 기반 성능 평가
n = X.shape[0]  # 표본 수
p = X.shape[1]  # 설명변수 수

# R²
cv_r2 = np.mean(cross_val_score(best_model, X, y, cv=10, scoring='r2'))

# Adjusted R²
adj_r2 = 1 - (1 - cv_r2) * (n - 1) / (n - p - 1)

# RMSE
neg_mse_scores = cross_val_score(best_model, X, y, cv=10, scoring='neg_mean_squared_error')
cv_rmse = np.mean(np.sqrt(-neg_mse_scores))

# MAE
neg_mae_scores = cross_val_score(best_model, X, y, cv=10, scoring='neg_mean_absolute_error')
cv_mae = np.mean(-neg_mae_scores)

# 3. 결과 출력
print("📊 교차검증 기반 성능 지표:")
print(f"R²            : {cv_r2:.5f}")
print(f"Adjusted R²   : {adj_r2:.5f}")
print(f"RMSE          : {cv_rmse:.5f}")
print(f"MAE           : {cv_mae:.5f}")

📊 교차검증 기반 성능 지표:
R²            : 0.72541
Adjusted R²   : 0.64220
RMSE          : 1604.23951
MAE           : 1122.66241


In [80]:
# 4. 전체 데이터로 모델 학습 (예측을 위해 필요)
best_model.fit(X, y)

# 5. 예측
df['예측_적정주차면수'] = best_model.predict(X)

# 6. 필요 주차면수 계산
df['필요주차면수'] = df['예측_적정주차면수'] - df['주차면수 합']

# 7. 우선순위 계산
df['우선순위지수'] = df['필요주차면수'] * (df['유입인구']/df['유입인구'].sum(axis = 0))
df['우선순위지수'] = (df['우선순위지수'] / df['우선순위지수'].max()).round(3)
df_sorted = df.sort_values(by='우선순위지수', ascending=False)

print("\n🏆 우선 설치 대상 상위 10개 행정동:")
print(df_sorted[['행정동', '필요주차면수', '우선순위지수']].head(10))


🏆 우선 설치 대상 상위 10개 행정동:
     행정동       필요주차면수  우선순위지수
25   인계동  8368.842773   1.000
37  광교1동  3106.172852   0.385
16  권선1동  3427.921387   0.270
27   매산동  3825.229980   0.264
41  영통3동  2445.466797   0.201
13    평동  2558.530273   0.200
36   원천동  1922.218750   0.196
31   행궁동  4378.890625   0.190
17   곡선동  3481.977539   0.185
14   서둔동  2520.435547   0.183


In [74]:
df_sorted[['행정동', '적정 주차면수', '예측_적정주차면수', '주차면수 합', '필요주차면수', '유입인구', '우선순위지수']]

,행정동,적정 주차면수,예측_적정주차면수,주차면수 합,필요주차면수,유입인구,우선순위지수
25,인계동,20823.00,16191.842773,7823,8368.842773,57072,455.500651
37,광교1동,8112.40,8259.172852,5153,3106.172852,59223,175.435114
16,권선1동,7467.54,7514.921387,4087,3427.921387,37583,122.863476
27,매산동,4748.23,5383.229980,1558,3825.229980,32910,120.056571
41,영통3동,8548.06,8355.466797,5910,2445.466797,39170,91.351534
13,평동,8724.35,8610.530273,6052,2558.530273,37331,91.087899
36,원천동,6887.66,6890.218750,4968,1922.218750,48780,89.422150
31,행궁동,7313.48,7459.890625,3081,4378.890625,20771,86.740517
17,곡선동,11512.45,10471.977539,6990,3481.977539,25418,84.404935
14,서둔동,10641.44,10065.435547,7545,2520.435547,34727,83.472489
